In [1]:
%%capture
# !pip3.12 install missingno
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import export_text
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Análisis preliminar


In [11]:
df_data_100 = pd.read_csv('train_data_100_lines.csv', encoding='ISO-8859-1')
df_label = pd.read_csv('train_labels_50_lines.csv', encoding='ISO-8859-1')

In [12]:
df_data_100.head()


,ID,Expenditure_AHF,Payment_6804,Infraction_CGP,Base_7744,Base_80863,Risk_1930,Expenditure_JIG,Infraction_SNZ,Base_02683,...,Infraction_WWLN,Infraction_ADWZ,Infraction_MZI,Infraction_QWWW,Infraction_YQXM,Infraction_QGR,Infraction_ZTLC,Infraction_LSX,Infraction_IBJ,Infraction_DNOU
0,3333337004103300505242400473433643475477705348...,2017-03-05,0.929122,0.382790,0.025782,1.002647,0.005515,0.089799,0.001479,0.005830,...,NaN,NaN,NaN,0.003776,0.001808,0.008200,NaN,0.001061,0.000027,0.002629
1,3333337004103300505242400473433643475477705348...,2017-04-20,0.918431,0.002224,0.006806,1.008186,0.003287,0.095966,0.007831,0.000861,...,NaN,NaN,NaN,0.004046,0.008288,0.001375,NaN,0.009999,0.002669,0.006912
2,3333337004103300505242400473433643475477705348...,2017-05-11,0.922563,0.567403,0.033713,1.007497,0.005594,0.094655,0.003162,0.010466,...,NaN,NaN,NaN,0.007742,0.002693,0.009851,NaN,0.000996,0.003254,0.004469
3,3333337004103300505242400473433643475477705348...,2017-06-29,0.919253,0.213193,0.031170,1.008534,0.008996,0.099089,0.009513,0.005360,...,NaN,NaN,NaN,0.002502,0.003399,0.004284,NaN,0.002642,0.004325,0.006308
4,3333337004103300505242400473433643475477705348...,2017-08-03,0.901860,0.325467,0.109644,1.005098,0.008041,0.108526,0.000732,0.005484,...,NaN,NaN,NaN,0.000275,0.007992,0.006918,NaN,0.009869,0.002979,0.005527


In [4]:
import pandas as pd

# Revisar si las transacciones de un mismo cliente están consecutivas
df_data_100['is_consecutive'] = df_data_100['ID'] == df_data_100['ID'].shift()

# Verificar si hay filas donde el ID del cliente no es consecutivo
non_consecutive = df_data_100[df_data_100['is_consecutive'] == False]

if non_consecutive.empty:
    print("Todas las transacciones de los clientes están colocadas consecutivamente.")
else:
    print(f"Hay {len(non_consecutive)} transacciones que no están consecutivas.")
    print(non_consecutive[['ID']].drop_duplicates())  # Muestra los IDs no consecutivos


Hay 9 transacciones que no están consecutivas.
                                                   ID
0   3333337004103300505242400473433643475477705348...
13  3333412724050477534287115505554450368233003786...
26  3333755354307801420025735367852078550468735254...
39  3333545238615133608512002405572210535308150267...
52  3333333216773173414240337455254307444633113087...
65  3331615143023053154650782333554783450473733165...
78  3331667757451082637147533444427274432044222417...
81  3331518360312256685203553502477321640385423445...
94  3331577436843324303562132281160853252523518165...


# Preprocesado 


In [ ]:
df_data = pd.read_csv('train_data.csv', encoding='ISO-8859-1')


In [9]:
df_data.shape[0]

3318787

### División en csv's


In [13]:

# Definir el tamaño objetivo de 500,000 filas por CSV
chunk_size = 500000

# Inicializar variables
current_chunk = []
current_chunk_size = 0
file_index = 1
last_customer_id = None

# Función para guardar el chunk actual en un CSV
def save_chunk(chunk, file_index):
    df_chunk = pd.DataFrame(chunk)
    df_chunk.to_csv(f'train_data_part_{file_index}.csv', index=False)
    print(f'Archivo train_data_part_{file_index}.csv guardado con {len(df_chunk)} filas.')

# Iterar sobre el DataFrame
for i, row in df_data.iterrows():
    customer_id = row['ID']

    # Si hemos alcanzado el tamaño máximo del chunk y el nuevo customer_id es diferente al último en el chunk
    if current_chunk_size >= chunk_size and customer_id != last_customer_id:
        save_chunk(current_chunk, file_index)  # Guardar el CSV actual
        file_index += 1
        current_chunk = []  # Reiniciar el chunk
        current_chunk_size = 0

    # Añadir la fila actual al chunk
    current_chunk.append(row)
    current_chunk_size += 1
    last_customer_id = customer_id

# Guardar el último chunk si no está vacío
if current_chunk:
    save_chunk(current_chunk, file_index)


Archivo train_data_part_1.csv guardado con 500011 filas.
Archivo train_data_part_2.csv guardado con 500003 filas.
Archivo train_data_part_3.csv guardado con 500004 filas.
Archivo train_data_part_4.csv guardado con 500004 filas.
Archivo train_data_part_5.csv guardado con 500001 filas.
Archivo train_data_part_6.csv guardado con 500008 filas.
Archivo train_data_part_7.csv guardado con 318756 filas.


In [14]:
df_data_part_1 = pd.read_csv('train_data_part_1.csv', encoding='ISO-8859-1')

In [17]:
pd.set_option('display.max_rows', None)

In [18]:
percentage = df_data_part_1.isnull().mean() * 100
columns_with_nulls = percentage[percentage > 0]
print(columns_with_nulls)


Payment_6804         0.861581
Base_80863           0.036399
Expenditure_JIG     18.512393
Infraction_SNZ       0.036399
Base_02683           0.036399
Infraction_SBF      85.429721
Infraction_ZWWJ     30.097338
Infraction_QJJF      4.999490
Infraction_EJZ       0.036399
Infraction_FMXQ     21.958917
Infraction_TLPJ     13.045513
Infraction_NCB      89.935821
Base_1165            0.002600
Base_6187            0.419391
Infraction_CLLY     56.598955
Infraction_AYWV      0.526988
Payment_22507        5.535278
Infraction_GWL      73.906974
Infraction_PAS       0.036399
Expenditure_HMO     18.512393
Infraction_BSU       3.396725
Infraction_GEL      54.170808
Base_14808           0.892580
Expenditure_KMW     52.999634
Infraction_HSSU      1.940357
Infraction_TBP      10.791163
Base_0580            0.111598
Infraction_PBC      13.748098
Infraction_DQLY      3.955113
Base_0229            0.036399
Base_8318           56.626954
Base_91828           0.036399
Infraction_HPS      88.570651
Base_6852 

## Cargado de csv's


In [2]:
for i in range(1, 8):
    file_name = f'train_data_part_{i}.csv'
    globals()[f'df_data_part_{i}'] = pd.read_csv(file_name, encoding='ISO-8859-1')
    print(f'Archivo {file_name} cargado en df_data_part_{i}')

Archivo train_data_part_1.csv cargado en df_data_part_1
Archivo train_data_part_2.csv cargado en df_data_part_2
Archivo train_data_part_3.csv cargado en df_data_part_3
Archivo train_data_part_4.csv cargado en df_data_part_4
Archivo train_data_part_5.csv cargado en df_data_part_5
Archivo train_data_part_6.csv cargado en df_data_part_6
Archivo train_data_part_7.csv cargado en df_data_part_7


### Comprobar columnas con valores nulos

In [46]:
def null_percentage(df, percentage):
    null_ratio = {}
    for col in df.columns:
        ratio = df[col].isna().sum() / len(df) * 100
        if ratio > percentage:
            null_ratio[col] = ratio
           
    return null_ratio

Me quedo solo con las columnas con menos de 80% de valores nulos

In [5]:
columns_with_high_nulls = []
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    null_columns = set(null_percentage(data_frame, 80).keys())
    
    # Si es el primer DataFrame, inicializar el conjunto con sus columnas
    if i == 1:
        columns_with_high_nulls = null_columns
    else:
        # Mantener solo las columnas que están en todos los DataFrames
        columns_with_high_nulls = columns_with_high_nulls.intersection(null_columns)

columns_with_high_nulls = list(columns_with_high_nulls)

print(f"Columnas con más del 80% de valores nulos en todos los CSVs: {columns_with_high_nulls}")

Columnas con más del 80% de valores nulos en todos los CSVs: ['Base_3958', 'Infraction_WWLN', 'Infraction_JVWF', 'Infraction_ZTLC', 'Infraction_IRKE', 'Infraction_QGC', 'Infraction_WLMI', 'Infraction_MZI', 'Risk_5797', 'Infraction_HPS', 'Infraction_HPLO', 'Infraction_SVKR', 'Base_64022', 'Infraction_ADWZ', 'Infraction_EBA', 'Risk_4561', 'Infraction_SBF', 'Infraction_APIU', 'Infraction_MAN', 'Infraction_WEG', 'Infraction_ANHZ', 'Base_8379', 'Infraction_NCB']


Eliminamos las columnas con más de 80% de valores nulos en todos los csv's

In [6]:
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame.drop(columns=columns_with_high_nulls, inplace=True)
    

### Comprobar si hay filas que tengan todas las columnas a null


In [8]:
# Verificar si hay filas completamente nulas
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    null_rows = data_frame[data_frame.isnull().all(axis=1)]
    if not null_rows.empty:
        print(f'Filas completamente nulas en df_data_part_{i}:')
        print(null_rows)
    else:
        print(f'No hay filas completamente nulas en df_data_part_{i}.')

No hay filas completamente nulas en df_data_part_1.
No hay filas completamente nulas en df_data_part_2.
No hay filas completamente nulas en df_data_part_3.
No hay filas completamente nulas en df_data_part_4.
No hay filas completamente nulas en df_data_part_5.
No hay filas completamente nulas en df_data_part_6.
No hay filas completamente nulas en df_data_part_7.


### Comprobar filas duplicadas

In [21]:
# Verificar si hay filas duplicadas en cada DataFrame
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    duplicate_rows = data_frame[data_frame.duplicated()]
    if not duplicate_rows.empty:
        print(f'Filas duplicadas en df_data_part_{i}:')
        print(duplicate_rows)
    else:
        print(f'No hay filas duplicadas en df_data_part_{i}.')

No hay filas duplicadas en df_data_part_1.
No hay filas duplicadas en df_data_part_2.
No hay filas duplicadas en df_data_part_3.
No hay filas duplicadas en df_data_part_4.
No hay filas duplicadas en df_data_part_5.
No hay filas duplicadas en df_data_part_6.
No hay filas duplicadas en df_data_part_7.


### Comprobar varianza 0

In [22]:
# Verificar si hay columnas con baja varianza
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    low_variance_cols = []
    for col in data_frame.columns:
        if data_frame[col].nunique() == 1:
            low_variance_cols.append(col)
    if low_variance_cols:
        print(f'Columnas con baja varianza en df_data_part_{i}: {low_variance_cols}')
    else:
        print(f'No hay columnas con baja varianza en df_data_part_{i}.')

No hay columnas con baja varianza en df_data_part_1.
No hay columnas con baja varianza en df_data_part_2.
No hay columnas con baja varianza en df_data_part_3.
No hay columnas con baja varianza en df_data_part_4.
No hay columnas con baja varianza en df_data_part_5.
No hay columnas con baja varianza en df_data_part_6.
No hay columnas con baja varianza en df_data_part_7.


Cuidado porque un 5% puede ser poco y a la vez, todas las filas de un cliente

In [47]:
# Calcular porcentajes de NaNs por columna teniendo en cuenta la intersección de los DataFrames y filtrando las columnas menores de 5% para eliminar las filas que tengan NaN en esa columna
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    percentage = data_frame.isnull().mean() * 100
    columns_with_nulls = percentage[percentage > 0]
    print(f'Porcentaje de valores nulos en df_data_part_{i}:')
    print(columns_with_nulls)
    print()


Valores nulos en df_data_part_1:
{'Payment_6804': np.float64(0.8615810452170053), 'Base_80863': np.float64(0.036399199217617216), 'Expenditure_JIG': np.float64(18.512392727359998), 'Infraction_SNZ': np.float64(0.036399199217617216), 'Base_02683': np.float64(0.036399199217617216), 'Infraction_ZWWJ': np.float64(30.097337858567112), 'Infraction_QJJF': np.float64(4.999490011219753), 'Infraction_EJZ': np.float64(0.036399199217617216), 'Infraction_FMXQ': np.float64(21.958916903828115), 'Infraction_TLPJ': np.float64(13.04551299871403), 'Base_1165': np.float64(0.002599942801258372), 'Base_6187': np.float64(0.4193907734029851), 'Infraction_CLLY': np.float64(56.598954822993896), 'Infraction_AYWV': np.float64(0.5269884062550624), 'Payment_22507': np.float64(5.535278223879074), 'Infraction_GWL': np.float64(73.90697404657097), 'Infraction_PAS': np.float64(0.036399199217617216), 'Expenditure_HMO': np.float64(18.512392727359998), 'Infraction_BSU': np.float64(3.396725272044015), 'Infraction_GEL': np.f

## Análisis de tipo de variable

In [23]:
# Agrupar columnas por tipo de dato
data_types = {}
# No hace falta comprobar todos los df ya que tienen las mismas columnas
data_frame = df_data_part_1
for col in data_frame.columns:
    data_type = str(data_frame[col].dtype)
    if data_type not in data_types:
        data_types[data_type] = []
    data_types[data_type].append(col)

for data_type, columns in data_types.items():
    print(f'{data_type}: {columns}')

object: ['ID', 'Expenditure_AHF', 'Infraction_YFSG', 'Infraction_DQLY', 'Infraction_CLH', 'Base_67254', 'Infraction_TEN']
float64: ['Payment_6804', 'Infraction_CGP', 'Base_7744', 'Base_80863', 'Risk_1930', 'Expenditure_JIG', 'Infraction_SNZ', 'Base_02683', 'Infraction_ZWWJ', 'Infraction_QJJF', 'Base_76065', 'Infraction_EJZ', 'Base_6872', 'Risk_0322', 'Infraction_FMXQ', 'Infraction_GGO', 'Infraction_TLPJ', 'Base_1165', 'Base_39598', 'Base_6187', 'Infraction_CLLY', 'Infraction_ZTNC', 'Base_85131', 'Risk_9995', 'Infraction_AYWV', 'Payment_22507', 'Base_9516', 'Infraction_GWL', 'Expenditure_YTR', 'Base_36384', 'Expenditure_FIP', 'Infraction_PAS', 'Risk_0003', 'Expenditure_HMO', 'Base_24406', 'Expenditure_LMSR', 'Infraction_BSU', 'Infraction_GEL', 'Base_14808', 'Risk_8065', 'Infraction_ZYW', 'Expenditure_KMW', 'Base_1039', 'Infraction_HSSU', 'Infraction_EHZP', 'Infraction_TBP', 'Base_0580', 'Expenditure_RGD', 'Infraction_PBC', 'Infraction_AQO', 'Base_0229', 'Base_8318', 'Base_69608', 'Base_

### Variables categóricas


In [26]:
# Hacemos print de las columnas de tipo object, solo primeras filas
data_frame = df_data_part_1
print(data_frame.select_dtypes(include='object').head())

                                                  ID Expenditure_AHF  \
0  3333337004103300505242400473433643475477705348...      2017-03-05   
1  3333337004103300505242400473433643475477705348...      2017-04-20   
2  3333337004103300505242400473433643475477705348...      2017-05-11   
3  3333337004103300505242400473433643475477705348...      2017-06-29   
4  3333337004103300505242400473433643475477705348...      2017-08-03   

  Infraction_YFSG Infraction_DQLY Infraction_CLH    Base_67254 Infraction_TEN  
0              CO               O      very_high  moderate_low  extremely_low  
1              CO               O      very_high  moderate_low  extremely_low  
2              CO               O      very_high  moderate_low  extremely_low  
3              CO               O      very_high  moderate_low  extremely_low  
4              CO               O      very_high  moderate_low  extremely_low  


####  

#### Transformación ID

In [27]:
# Pasamos ID a número en todos los dataframes para facilitar su procesado
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['ID'] = pd.to_numeric(data_frame['ID'], errors='coerce')

#### Transformación Expenditure_AHF

In [28]:
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['Expenditure_AHF'] = pd.to_datetime(data_frame['Expenditure_AHF'], errors='coerce')
    data_frame['Expenditure_AHF_year'] = data_frame['Expenditure_AHF'].dt.year
    data_frame['Expenditure_AHF_month'] = data_frame['Expenditure_AHF'].dt.month
    data_frame['Expenditure_AHF_day'] = data_frame['Expenditure_AHF'].dt.day
    data_frame.drop(columns=['Expenditure_AHF'], inplace=True)


#### Transformación Infraction_YFSG

In [30]:
# Ver los valores de la columna Infraction_YFSG en todos los DataFrames y hacer una intersección
unique_values = set()
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    unique_values_i = set(data_frame['Infraction_YFSG'].unique())
    if not unique_values:
        unique_values = unique_values_i
    else:
        unique_values = unique_values.intersection(unique_values_i)

print(unique_values)

{'XZ', 'CL', 'XM', 'XL', 'CO', 'CR'}


In [31]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['Infraction_YFSG_encoded'] = label_encoder.fit_transform(data_frame['Infraction_YFSG'])
    data_frame.drop(columns=['Infraction_YFSG'], inplace=True)


#### Transformación Infraction_DQLY

In [32]:
# Ver los valores de la columna Infraction_YFSG en todos los DataFrames y hacer una intersección
unique_values = set()
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    unique_values_i = set(data_frame['Infraction_DQLY'].unique())
    if not unique_values:
        unique_values = unique_values_i
    else:
        unique_values = unique_values.intersection(unique_values_i)

print(unique_values)

{'R', nan, 'U', '-1', 'O'}


In [37]:
# Análisis de porcentaje de NaNs en cada DataFrame en la columna Infraction_DQLY
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    print(f'Porcentaje de NaNs en df_data_part_{i}: {data_frame["Infraction_DQLY"].isna().mean() * 100:.2f}%')

Porcentaje de NaNs en df_data_part_1: 3.96%
Porcentaje de NaNs en df_data_part_2: 3.88%
Porcentaje de NaNs en df_data_part_3: 3.98%
Porcentaje de NaNs en df_data_part_4: 4.02%
Porcentaje de NaNs en df_data_part_5: 3.94%
Porcentaje de NaNs en df_data_part_6: 3.84%
Porcentaje de NaNs en df_data_part_7: 3.93%


Antes de codificarlo necesitamos tratar los NaN

In [41]:
# Como el porcentaje de NaNs es muy bajo (aprox. 4%), rellenamos los NaNs con el valor más común
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    most_common_value = data_frame['Infraction_DQLY'].mode()[0]
    data_frame['Infraction_DQLY'].fillna(most_common_value, inplace=True)



C:\Users\eleni\AppData\Local\Temp\ipykernel_30512\2308713897.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_frame['Infraction_DQLY'].fillna(most_common_value, inplace=True)


##### Hacemos la codificación de la columna

In [44]:
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    data_frame['Infraction_DQLY_encoded'] = label_encoder.fit_transform(data_frame['Infraction_DQLY'])
    data_frame.drop(columns=['Infraction_DQLY'], inplace=True)


#### Transformación Infraction_CLH

In [45]:
# Ver los valores de la columna Infraction_YFSG en todos los DataFrames y hacer una intersección
unique_values = set()
for i in range(1, 8):
    data_frame = globals()[f'df_data_part_{i}']
    unique_values_i = set(data_frame['Infraction_CLH'].unique())
    if not unique_values:
        unique_values = unique_values_i
    else:
        unique_values = unique_values.intersection(unique_values_i)

print(unique_values)

{'moderate', 'moderate_low', nan, 'very_high', 'moderate_high', 'very_low', 'low', 'high'}


Antes de codificarlo, tratamos los NaN

## Comprobación NaNs **************

## Tratar valores infinitos

## Selección de características


## Split Dataset